# First order ARMAX model with Stan

This notebook trains a first order ARMAX model to predict the energy consumption observed by one of the 4 meters installed in a building.

The parameters of the model are learned by Bayesian inference with Stan, then the predictions are made outside of the Stan model, in Python.

The notebook is mostly undocumented for now.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Stan
import stan

# %% Reading the data file
df = pd.read_csv('data/building_1298.csv')
df.set_index(pd.to_datetime(df['datetime']), inplace=True)
df.fillna(method='ffill', inplace=True)

# Adding the day of the week as a new feature in the dataset
df['hour'] = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['weekend'] = ((df.index.weekday == 5) | (df.index.weekday == 6))

# %% First order ARMAX model
model_code = """
data {
  int<lower=0> T_train;     // data points (training)
  int<lower=0> K;       // number of predictors

  matrix[T_train, K] x;       // predictor matrix (training and test)
  vector[T_train] y;    // output data
}

parameters {
  real mu;           // mean coeff
  real phi;          // AR
  real theta;        // MA
  real theta_x[K];   // X
  real<lower=0> sigma;
}

model {
  vector[T_train] nu;             // prediction for time t
  vector[T_train] err;            // error for time t
  
  // priors
  // phi ~ uniform(0, 1);

  // initialisation
  nu[1] = y[1];
  err[1] = 0;

  // the rest of the time steps
  for (t in 2:T_train) {
    real ex = 0;
    for (j in 1:K)
      ex += theta_x[j] * x[t, j];
    nu[t] = mu + phi * y[t-1] + theta * err[t-1] + ex;
    err[t] = y[t] - nu[t];
    }

   err ~ normal(0, sigma);  // likelihood is here
}
"""



# %% Setting up the modelling boundaries

# Choosing the period for training and test data
training_start = '2016-03-01 00:00:00'
training_end = '2016-03-31 23:00:00'
test_end = '2016-04-07 23:00:00'
df = df.drop(df.index[(df.index < pd.to_datetime(training_start)) | (df.index > pd.to_datetime(test_end))])
training_mask = df.index <= pd.to_datetime(training_end)

# Choosing the input and output features of the model
# x = df[['tite', 'i_sol', 'tits']]
x = df[['air_temperature']]
y = df['m0']
x_train = x.iloc[training_mask]
y_train = y.iloc[training_mask]

arx_data = {'T_train': len(y_train),
            'K': np.shape(x_train)[1],
            'x': x_train.values,
            'y': y_train.values}

#%% Learning

init0 = [{"mu":300}, {"phi":0.6}, {"theta":-0.4}, {"sigma":50}]     # for meter 0
init1 = [{"mu":800}, {"phi":0.8}, {"sigma":500}]                    # for meter 1
control = {'adapt_delta': 0.9, 'max_treedepth': 15}

posterior = stan.build(model_code, data=arx_data)
fit = posterior.sample(num_chains=4, num_samples=1000, init=init0)
df_post = fit.to_frame()

# %% Calculating the information criteria
P = 1
Q = 1
K = 1
T = len(y_train)

logL = df_post['lp__'].max()
AIC = -2 * logL + 2 * (P + Q + K + 1)
AICc = AIC + 2 * (P + Q + K + 1) * (P + Q + K + 2) / (T - P - Q - K - 2)

# %% Predictions

def prediction(sample):
    mu = df_post['mu'][sample]
    phi = df_post['phi'][sample]
    theta = df_post['theta'][sample]
    theta_x = df_post.loc[sample, df_post.columns.str.contains('theta_x')]
    sigma = df_post['sigma'][sample]

    y_new = np.zeros(len(y))
    err_new = np.zeros(len(y))

    # initialisation
    y_new[0] = y_train[0]
    err_new[0] = 0

    for t in range(P, len(y_train)):
        ex = 0
        for j in range(0, np.shape(x)[1]):
            ex += theta_x[j] * x.iloc[t, j]
        y_new[t] = mu + phi * y_train.iloc[t-1] + theta * err_new[t-1] + ex
        err_new[t] = y_train.iloc[t] - y_new[t]
    # all time steps that do not have observations
    for t in range(len(y_train), len(y)):
        ex = 0
        for j in range(0, np.shape(x)[1]):
            ex += theta_x[j] * x.iloc[t, j]
        y_new[t] = mu + phi * y_new[t-1] + theta * err_new[t-1] + ex
        err_new[t] = 0

    return y_new, err_new


# %% Plotting the predictions over the test period

t_ = df.index
teal = '#029386'
orange = '#F97306'

fig, ax = plt.subplots(figsize=(8, 4))
ax.plot(t_, y, color=teal, label='Observation')

essais = 50
y_new = np.zeros((essais, len(df)))
err_new = np.zeros((essais, len(df)))
for i in range(essais):
    sample = np.random.randint(0, len(df_post['mu']))
    y_new[i], err_new[i] = prediction(sample)
    ax.plot(t_, y_new[i], color=orange, alpha=0.2)
# ax.fill_between(t_, y_pred_avg-2*y_pred_std, y_pred_avg+2*y_pred_std, color=orange, alpha=0.3)
ax.set_ylabel('Heat pump energy use (Wh/15min)')
ax.legend()
# ax.set_xlim([pd.to_datetime(training_end), pd.to_datetime(test_end)])
plt.show()


RuntimeError: asyncio.run() cannot be called from a running event loop